<a href="https://colab.research.google.com/github/navavatpi/researchPhd/blob/developANN/3_dataModelingMLPModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. Data Modeling: MLP model

## Environment Setup

## TPU backend

In [1]:
# # Enabling and testing the TPU
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # install scikit-plot to Colab
# !pip install scikit-plot

## GPU backend

In [2]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# # install scikit-plot to Colab
# !pip install scikit-plot

In [3]:
# libraries import
import os # miscellaneous operating system interfaces
import numpy as np # Array library for linear algebra
import matplotlib.pyplot as plt # visualization library
from sklearn.metrics import classification_report # evaluators classification report
import scikitplot as skplt # scikit-learn and matplotlib integrated visualization library
from tensorflow import keras # back-end artificial neural network API
from tensorflow.keras import layers # layer API for neural network architechture
from tensorflow.keras.optimizers import Adam # optimizer in neural network implement
from tensorflow.keras.callbacks import ReduceLROnPlateau # Reduce learning rate when a metric has stopped improving
from keras.models import model_from_json # loading model in JSON format

# system configuration
os.environ["TF_CPP_MIN_LOG_LEVEL"]="0" # disable warning messages
os.environ['AUTOGRAPH_VERBOSITY'] = '0' # set it to 0
random_seed = 20 # random seed

print('\n')
print('necessary libraries has successfully imported')



necessary libraries has successfully imported


In [4]:
# connect to google drive to obtain dataset
# !note: need to perform authorization by following below link
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# define model name
modelName = 'MLP'

print(modelName, 'modeling is initializing')
print('\n')

MLP modeling is initializing




## Data Loading

In [6]:
# define path to files
defaultdir = '/content/drive/My Drive/all_about_phd_research/'

In [7]:
print('loading processed data')
print('\n')

# load input data from drive
X_train = np.load(defaultdir + '/processed_data/X_train.npy')
X_test = np.load(defaultdir + '/processed_data/X_test.npy')
y_train = np.load(defaultdir + '/processed_data/y_train.npy')
y_test = np.load(defaultdir + '/processed_data/y_test.npy')

print('loading done')
print('\n')

loading processed data


loading done




## Data Modeling

In [8]:
# define function to create MLP model
def mlpModel(data_train, num_classes, layer=1, units= 64, 
             activation= "relu", use_bias=True, drop_size=0.5, 
             ):
    
    # input layer convert arrays => tensors
    inputs = keras.Input(data_train.shape[1:]) # dimension of the input

    x1 = layers.Dense(units = units, 
                      activation= activation,
                      use_bias= use_bias)(inputs)
    x1 = layers.Dense(units = units, 
                      activation= activation,
                      use_bias= use_bias)(x1)
    x1 = layers.Dropout(drop_size)(x1)
    
    dense_size = units
    
    if layer > 1:
        
        x1 = layers.Dense(units = 2*units, 
                          activation= activation,
                          use_bias= use_bias)(x1)
        x1 = layers.Dense(units = 2*units, 
                          activation= activation,
                          use_bias= use_bias)(x1)
        x1 = layers.Dropout(drop_size)(x1)
        
        dense_size = 2*units
        
        if layer > 2:

            x1 = layers.Dense(units = 4*units, 
                              activation= activation,
                              use_bias= use_bias)(x1)
            x1 = layers.Dense(filunitsters = 4*units, 
                              activation= activation,
                              use_bias= use_bias)(x1)
            x1 = layers.Dropout(drop_size)(x1)
            
            dense_size = 4*units
            
            if layer > 3:

                x1 = layers.Dense(units = 8*units, 
                                  activation= activation,
                                  use_bias= use_bias)(x1)
                x1 = layers.Dense(units = 8*units, 
                                  activation= activation,
                                  use_bias= use_bias)(x1)
                x1 = layers.Dropout(drop_size)(x1)
                
                dense_size = 8*units
                
                if layer > 4:

                    x1 = layers.Dense(units = 8*units, 
                                      activation= activation,
                                      use_bias= use_bias)(x1)
                    x1 = layers.Dense(units = 8*units, 
                                      activation= activation,
                                      use_bias= use_bias)(x1)
                    x1 = layers.Dropout(drop_size)(x1)
                    
                    dense_size = 8*units

    x2 = layers.Flatten()(x1)
    x2 = layers.Dense(dense_size, activation = activation)(x2)
    # x2 = layers.Dense(dense_size, activation = activation)(x1)

    outputs = layers.Dense(num_classes, activation= "softmax")(x2)

    mlpModel = keras.Model(inputs=inputs, outputs=outputs, name="mlpModel")

    mlpModel.summary()
    
    return mlpModel

# !!!warning: if assign layer > 1 will causing OOM (Out Of Memory) error

In [ ]:
# perform MLP model define function
mlpModel = mlpModel(X_train, layer=1, num_classes=3)

print('\n')
print(modelName, 'model contruction is done')
print('\n')

In [ ]:
# plot MLP model architecture
# the figure will be saved in the same directory with input datas
tf.keras.utils.plot_model(mlpModel, 
                          to_file=defaultdir + '/processed_data/{}_model_architecture.png'.format(modelName), 
                          show_shapes=True)

In [ ]:
# define ADAM opimizer & complie resulting in ready-to-train model function
def modelComplie(mlpModel, X_train, y_train, 
             X_test, y_test,
             epochs, batch_size, 
             learning_rate=0.001, beta_1=0.9, beta_2=0.999, 
             epsilon=1e-08, decay=0):

  # Define the optimizer
  optimizer = Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, 
                   epsilon=epsilon, decay=decay)

  # assign metrics for the model
  mlpModel.compile(optimizer=optimizer , 
                   loss="categorical_crossentropy", 
                   metrics=["accuracy"])

## Model Training

In [ ]:
# define training epoch number & batch size  parameters
epochs_list = [5]
batch_size_list = [5]

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                          patience=3, 
                                          verbose=1, 
                                          factor=0.5, 
                                          min_lr=0.00001)
  
# loop for training model
for epochs in epochs_list:
  for batch_size in batch_size_list:

    # perform compile model function
    modelComplie(mlpModel=mlpModel, X_train=X_train, y_train=y_train, 
         X_test=X_test, y_test=y_test, 
         epochs=epochs, batch_size=batch_size)
    
    print('\n')
    print(modelName, 'model is training in total =', epochs, 'epochs')
    print('In each epoch has batch size =', batch_size, 'samples')
    print('\n')
    
    
    # training model with learning rate annealer
    history = mlpModel.fit(X_train, y_train, 
                         batch_size=batch_size,
                         epochs = epochs, 
                         validation_data = (X_test,y_test),
                         verbose = 1, 
                         steps_per_epoch=X_train.shape[0]//batch_size,
                         callbacks=[learning_rate_reduction])
    
    print('\n')
    print(modelName,'model training has done')
    print('\n')

## Model Prediction
### Note:
At this stage, we have 5 datasets and 1 model which are
1. X_train: training dataset
2. X_test: testing dataset
3. y_train: training label
4. y_test: testing label
5. y_pred: prediction label from model
6. mlpModel: MLP model

In [ ]:
print(modelName, 'model is making prediction')
print('\n')

# making predictions
y_pred = mlpModel.predict(X_test, verbose=1)

print('\n')
print(modelName, 'model making prediction has done')
print('\n')

# Model Evaluation

In [ ]:
print(modelName, 'model evaluation is now processing')
print('\n')

In [ ]:
print('loss & accuracy plot')
print('\n')

# Plot the loss & accuracy curves for training and validation
# Define number of plots -> 2
figurename = 'Loss and Accuracy plot of {} model'.format(modelName)

plt.figure(figsize=(12,12))
plt.tight_layout()

# set font sizes
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

# -------------------------------------------------------------------------------------------------------------------------------
# Loss plot
plt.subplot(2,1,1)
plt.plot(history.history['loss'], '--', color='b', label="training loss")
plt.plot(history.history['val_loss'], '-', color='b', label="validation loss")

# decorating
plt.axis([0, 1, 0, 1]) # define lower & upper boundaries of axes ([x-axis, y-axis])
legend = plt.legend(loc='center right', shadow=True, bbox_to_anchor=(1.375, 0.5), ncol=1,fontsize=MEDIUM_SIZE)
plt.title('Loss plot', fontsize=BIGGER_SIZE)
plt.tick_params(axis="x", labelsize=15)
plt.tick_params(axis="y", labelsize=15)

# -------------------------------------------------------------------------------------------------------------------------------
# Accuracy plot
plt.subplot(2,1,2)
plt.plot(history.history['accuracy'], '--', color='r', label="training accuracy")
plt.plot(history.history['val_accuracy'], '-', color='r',label="validation accuracy")

# decorating
plt.axis([0, 1, 0, 1]) # define lower & upper boundaries of axes ([x-axis, y-axis])
legend = plt.legend(loc='lower right', shadow=True, bbox_to_anchor=(1.43, 0.5), ncol=1, fontsize=MEDIUM_SIZE)
plt.title('Accuracy plot', fontsize=BIGGER_SIZE)
plt.tick_params(axis="x", labelsize=15)
plt.tick_params(axis="y", labelsize=15)

# plt.subplots_adjust(hspace=0.5)

# save figure
plt.savefig(defaultdir + '/processed_data/' + figurename + '.png')

## Confusion matrix and AUC-ROC curve

In [ ]:
# def plot confusion matrix function
def plotConfusionMatrix(y_test=y_test, y_pred=y_pred, cmap=plt.cm.Reds, title='Confusion Matrix of {} model'.format(modelName)):

  # set font sizes
  SMALL_SIZE = 12
  MEDIUM_SIZE = 15
  BIGGER_SIZE = 20

  skplt.metrics.plot_confusion_matrix(y_true=np.argmax(y_test, axis=1), 
                                      y_pred=np.argmax(y_pred, axis=1), 
                                      title=title, 
                                      normalize='true', 
                                      figsize=(9,9), 
                                      cmap=cmap, 
                                      title_fontsize= BIGGER_SIZE, 
                                      text_fontsize= BIGGER_SIZE
                                      )

  # save figure
  plt.savefig(defaultdir + '/processed_data/' + title + '.png')

In [ ]:
print('confusion matrix plot')
print('\n')

# perform plot confusion matrix function
title = 'Confusion Matrix of {} model'.format(modelName)
plot = plotConfusionMatrix(y_test=y_test, y_pred=y_pred, cmap=plt.cm.Reds, title=title)

In [ ]:
# def plot ROC curve function
def plotROC(y_test=y_test, y_pred=y_pred, cmap=plt.cm.Reds, title='ROC curve of {} model'.format(modelName)):

  # set font sizes
  SMALL_SIZE = 12
  MEDIUM_SIZE = 15
  BIGGER_SIZE = 20

  skplt.metrics.plot_roc(y_true=np.argmax(y_test, axis=1), 
                                      y_probas=y_pred, 
                                      title=title, 
                                      plot_micro=True, 
                                      plot_macro=True, 
                                      figsize=(12,6), 
                                      cmap=cmap, 
                                      title_fontsize= BIGGER_SIZE, 
                                      text_fontsize= MEDIUM_SIZE
                         )

  # save figure
  plt.savefig(defaultdir + '/processed_data/' + title + '.png')

In [ ]:
print('AUC-ROC curve plot')
print('\n')

# perform plot AUC-ROC curve function
title = 'AUC-ROC curve of {} model'.format(modelName)
plot = plotROC(y_test=y_test, y_pred=y_pred, cmap=plt.cm.Reds, title=title)

## Print Classification Report

In [ ]:
# classification report
num_classes = 3
class_list = ['class '+ str(y_class) for y_class in range(num_classes)]

print('classification report of {} model'.format(modelName))
print('\n')
print(classification_report(np.argmax(y_test,axis=1), 
                            np.argmax(y_pred,axis=1), 
                            target_names= class_list))

print('\n')

## Saving model

In [ ]:
# define saving function
def save_model (model, model_name):

    # serialize model to JSON
    model_json = model.to_json()
    
    with open("{}.json".format(defaultdir + '/processed_data/' + model_name), "w") as json_file:
        
        json_file.write(model_json)

        # serialize weights to HDF5
        model.save_weights("{}_weight.h5".format(defaultdir + '/processed_data/' + model_name))

    print("Saved {} to drive".format(model_name))

In [ ]:
# perform saving
save_model(model=mlpModel, model_name= modelName.lower() + 'Model')

In [ ]:
import sys
sys.getsizeof(mlpModel.get_weights())

## Load the saved model

In [ ]:
# define loading function
def load_model (model, model_name):

    # load json and create model
    json_file = open('{}.json'.format(defaultdir + '/processed_data/' + model_name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights("{}_weight.h5".format(defaultdir + '/processed_data/' + model_name))
    print("Loaded {} from drive".format(model_name))
    
    return loaded_model

In [ ]:
# perform loading
mlpModel = load_model(model=mlpModel, model_name= modelName.lower() + 'Model')

## Perform testing loaded model

In [ ]:
# perform compile model function
modelComplie(mlpModel=mlpModel, X_train=X_train, y_train=y_train, 
         X_test=X_test, y_test=y_test, 
         epochs=epochs, batch_size=batch_size)

# evaluate the loaded model
mlpModel.evaluate(X_train, y_train, verbose=1)

In [ ]:
# making prediction of loaded model
mlpModel.predict(X_test, verbose=1)

In [ ]:
print('\n')
print(modelName, 'modeling process is completed')

# Finish Data life cycle